<a href="https://colab.research.google.com/github/yangxiaoming5201/hello-world/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf


In [ ]:
!pwd

/content


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
root_path='/content/data/AirQ'#数据存放路径

In [ ]:
site_list = os.listdir(root_path)
file_list = [os.path.join(root_path, file) for file in site_list]
file_list

['/content/data/AirQ/1001A_AirQ.xls',
 '/content/data/AirQ/1010A_AirQ.xls',
 '/content/data/AirQ/1002A_AirQ.xls',
 '/content/data/AirQ/1005A_AirQ.xls',
 '/content/data/AirQ/1006A_AirQ.xls',
 '/content/data/AirQ/1009A_AirQ.xls',
 '/content/data/AirQ/1008A_AirQ.xls',
 '/content/data/AirQ/1011A_AirQ.xls',
 '/content/data/AirQ/1007A_AirQ.xls',
 '/content/data/AirQ/1003A_AirQ.xls',
 '/content/data/AirQ/1004A_AirQ.xls']

In [ ]:
# 读取所有数据
data_list=[]
for file in file_list:
    data = pd.read_excel(file,
                         usecols=[0, 1, 2, 3, 4, 6, 8, 12, 16],
                         index_col=[0, 1, 2])
    data_list.append(data)
data_list

[                               AQI  PM2.5  PM10  SO2   O3   CO
 date     day_of_the_week hour                                 
 20140601 0               0      90     67   378    2  136  0.8
                          1      88     65   378    2  131  0.7
                          2      83     61    74    2  118  0.7
                          3      68     49    74    2  143  0.6
                          4      60     43    74    2  135  0.6
 ...                            ...    ...   ...  ...  ...  ...
 20200630 2               19     48     10    10    2  152  0.4
                          20     45     17    45    2  140  0.4
                          21     38     18    35    3  120  0.4
                          22     36     17    36    3  107  0.4
                          23     54     16    57    2   95  0.4
 
 [53328 rows x 6 columns],
                                AQI  PM2.5  PM10  SO2   O3   CO
 date     day_of_the_week hour                                 
 20140601 0

In [ ]:
np.array(data_list,dtype=float)

array([[[9.00e+01, 6.70e+01, 3.78e+02, 2.00e+00, 1.36e+02, 8.00e-01],
        [8.80e+01, 6.50e+01, 3.78e+02, 2.00e+00, 1.31e+02, 7.00e-01],
        [8.30e+01, 6.10e+01, 7.40e+01, 2.00e+00, 1.18e+02, 7.00e-01],
        ...,
        [3.80e+01, 1.80e+01, 3.50e+01, 3.00e+00, 1.20e+02, 4.00e-01],
        [3.60e+01, 1.70e+01, 3.60e+01, 3.00e+00, 1.07e+02, 4.00e-01],
        [5.40e+01, 1.60e+01, 5.70e+01, 2.00e+00, 9.50e+01, 4.00e-01]],

       [[8.30e+01, 6.10e+01, 4.08e+02, 2.00e+00, 1.33e+02, 7.00e-01],
        [6.40e+01, 4.60e+01, 4.08e+02, 2.00e+00, 1.35e+02, 7.00e-01],
        [5.20e+01, 3.60e+01, 4.08e+02, 2.00e+00, 1.42e+02, 6.00e-01],
        ...,
        [4.40e+01, 1.90e+01, 4.40e+01, 1.00e+00, 1.15e+02, 4.00e-01],
        [3.40e+01, 1.40e+01, 3.40e+01, 3.00e+00, 1.08e+02, 3.00e-01],
        [3.30e+01, 1.10e+01, 2.70e+01, 3.00e+00, 1.05e+02, 3.00e-01]],

       [[1.65e+02, 1.25e+02, 1.68e+02, 2.60e+01, 2.37e+02, 2.00e+00],
        [1.65e+02, 1.25e+02, 1.68e+02, 2.60e+01, 2.37e+02, 2

In [ ]:
def standardization(data_list1):
    """
    将数据标准化
    """
    ss = StandardScaler()
    std_parmas_list = []  #标准化的相关参数,方便反标准化(均值/标准差)
    std_data_list = []  #标准化后的数据列表
    for data in data_list1:
        #     print(ss.fit_transform(data))
        fit = ss.fit(data)
        parmas = {"mean": ss.mean_, "scale": ss.scale_}
        std_parmas_list.append(parmas)
        std_data_list.append(ss.transform(data))
    return std_data_list, std_parmas_list


std_data_list, std_parmas_list = standardization(data_list)
std_data_list

[array([[-0.05309848,  0.04352086,  3.22877847, -0.49473473,  1.42823815,
         -0.16496717],
        [-0.07849865,  0.01494862,  3.22877847, -0.49473473,  1.33628001,
         -0.22435709],
        [-0.14199907, -0.04219586, -0.24792589, -0.49473473,  1.09718884,
         -0.22435709],
        ...,
        [-0.71350282, -0.65649902, -0.69395047, -0.41822483,  1.1339721 ,
         -0.40252685],
        [-0.73890298, -0.67078514, -0.68251394, -0.41822483,  0.89488093,
         -0.40252685],
        [-0.51030148, -0.68507126, -0.44234686, -0.49473473,  0.67418139,
         -0.40252685]]),
 array([[ 0.01046988,  0.13364296,  4.26075048, -0.49979752,  1.32527002,
         -0.13226677],
        [-0.26606475, -0.12855708,  4.26075048, -0.49979752,  1.3622124 ,
         -0.13226677],
        [-0.4407182 , -0.30335711,  4.26075048, -0.49979752,  1.49151076,
         -0.18956822],
        ...,
        [-0.55715384, -0.60051717, -0.5097184 , -0.59238451,  0.99278854,
         -0.30417113],
  

In [ ]:
#反标准化
def anti_standardization(data_list, parmas_list):
    for data, parmas in zip(data_list, std_parmas_list):
        ss = StandardScaler()
        ss.mean_ = parmas['mean']
        ss.scale_ = parmas['scale']
        print(ss.inverse_transform(data))

# anti_standardization(std_data_list, std_parmas_list)

In [ ]:
def look_list_shape(look_list):
    """
    查看list的维度
    """
    print(np.array(look_list).shape)


look_list_shape(std_data_list)

(11, 53328, 6)


In [ ]:
def split_dataset(dataset_list, train_rate=0.8, valid_rate=0.25):
    """
    划分数据集
    """
    for dataset in dataset_list:
        li = train_test_split(dataset)
        print(look_list_shape(li[0]),look_list_shape(li[1]),'-----------\n')
split_dataset(std_data_list)

(39996, 6)
(13332, 6)
None None -----------

(39996, 6)
(13332, 6)
None None -----------

(39996, 6)
(13332, 6)
None None -----------

(39996, 6)
(13332, 6)
None None -----------

(39996, 6)
(13332, 6)
None None -----------

(39996, 6)
(13332, 6)
None None -----------

(39996, 6)
(13332, 6)
None None -----------

(39996, 6)
(13332, 6)
None None -----------

(39996, 6)
(13332, 6)
None None -----------

(39996, 6)
(13332, 6)
None None -----------

(39996, 6)
(13332, 6)
None None -----------



In [ ]:
53328/24

2222.0